In [1]:
import numpy as np
import scipy as sp
import sympy as sy
import json 
import h5py

import SimulatorParser as sim_par
import rates as rates_module

In [2]:

reactions_file = "reactionsSimple.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

parser = sim_par.SimulatorParser(reactions_file, const_dict)

output_parser = parser.create_physical_model()


dO_F/dt = -0.02*O_F*O_S*r_7 - 1.0*O_F*r_2 - 1.0*O_F*r_4 - 0.02*O_F*r_5*(1.0 - O_S) + 1.0*r_1*(1.0 - O_F)
dO_S/dt = -1.0*O_F*O_S*r_7 + 1.0*O_F*r_5*(1.0 - O_S) - 1.0*O_S*r_6 + 1.0*r_3*(1.0 - O_S)


In [3]:
print(output_parser.keys())

dict_keys(['model', 'species_model', 'rates_model', 'reactions_list'])


In [4]:

####* obtain the experimental data structure

def prepare_data(file_exp_data):
    #### the .hdf5 file was obtained by merging the data from the .xlsx files
    ### Read the experimental data from the hdf5 file
    data_dict = {}
    with h5py.File(file_exp_data, "r") as file:
        keys = list(file.keys())
        
        for key in keys:
            data_dict[key] = file[key][:]
        
    file.close()
        
    ####* Prepara the data for the simulation
    Pressure_vec = data_dict["Pressure"]
    Current_vec = data_dict["Current"]
    
    Tnw_vec = data_dict["TnwExp"]
    Tw_vec = data_dict["Twall"]
    
    zeros = np.zeros_like(Tnw_vec)
    Oden_vec  = data_dict.get("OmeanExp", zeros.copy())
    COden_vec = data_dict.get("COmeanExp",zeros.copy())
    Nexp_vec  = data_dict.get("NExp", zeros.copy())
    
    Recomb_prob_exp_vec = data_dict["recProbExp"]
    FluxIon_vec = 1e14 * Current_vec        
    
    ####? EavgMB data
    p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
    EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
    interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)
    
    exp_vec = []
    for i in range(len(Tnw_vec)):
        exp_vec.append({
                "pressure": Pressure_vec[i], "current": Current_vec[i],
                "Tnw": Tnw_vec[i], "Tw": Tw_vec[i], 
                "O": Oden_vec[i], "CO": COden_vec[i], "N": Nexp_vec[i],
                "Ion": FluxIon_vec[i], "EavgMB": interpolator(Pressure_vec[i]).item()
        })
        
    exp_vec_arr = np.array(exp_vec, dtype=object)
    return exp_vec_arr, Recomb_prob_exp_vec


exp_vec_arr, Recomb_prob_exp_vec = prepare_data("Experimental_data_Paper.hdf5")



In [5]:
exp_dict = exp_vec_arr[0]

reactions_list = output_parser['reactions_list']

print(exp_dict)


flux_species_list_aux = [reaction['gas_specie'] for reaction in  reactions_list]
flux_species_list = list(set([item for line in flux_species_list_aux for item in line]))
print(flux_species_list)


###! update exp dict with the flux calculation insert formula here
for ele in flux_species_list:
    exp_dict[ele] = 2.0*exp_dict[ele]


{'pressure': 0.4, 'current': 20.0, 'Tnw': 262.53133733208, 'Tw': 253.14999999999998, 'O': 1493906152913230.0, 'CO': 0.0, 'N': 1.34810445111836e+16, 'Ion': 2000000000000000.0, 'EavgMB': 0.87}
['O']


In [6]:

####* more inteliget way of doing rate calculation

different_rates_list = list(set([reaction['rate'] for reaction in reactions_list]))
available_rates_functions = dict()

for name in different_rates_list:
    if hasattr(rates_module, name):
        available_rates_functions[name] = getattr(rates_module, name)
    else:
        raise ValueError(f"{name} is not defined on the rates module!")


print(available_rates_functions)


rates_calculation = []

for i in range(len(reactions_list)):
    
    model_dict = reactions_list[i]['model_dict']
    
    ### manipulation of the specif parameters here 
    
    gas_specie = reactions_list[i]['gas_specie']
    model_dict['gas_specie'] = gas_specie
    
    rate_name = reactions_list[i]['rate']
    rate_fun = available_rates_functions[rate_name]
    rate_value = rate_fun(const_dict, exp_dict, model_dict)
    rates_calculation.append(rate_value)


print(rates_calculation)

{'recomb_LH': <function recomb_LH at 0x1177eade0>, 'diffusion': <function diffusion at 0x1177eaca0>, 'adsorption': <function adsorption at 0x1177eab60>, 'desorption': <function desorption at 0x1177eac00>, 'recomb_ER': <function recomb_ER at 0x1177ead40>}
[1.95281850054017, 645547731.6715667, 1.95281850054017, 0.002024065544015633, 1000000000000000.0, 0.002024065544015633, 803459850690.479]


In [7]:


# rates_calculation = []


# for i in range(len(reactions_list)):

#     model_dict = reactions_list[i]['model_dict']
#     gas_specie = reactions_list[i]['gas_specie']
#     model_dict['gas_specie'] = gas_specie

#     rate_name = reactions_list[i]['rate']
#     if hasattr(rates,rate_name):
#         rate_fun = getattr(rates, rate_name)
#         rate_value = rate_fun(const_dict, exp_dict, model_dict)
#         rates_calculation.append(rate_value)
#     else:
#         raise ValueError(f"Rate of the reaction {reactions_list[i]['id']} - {reactions_list[i]["rate"]} - is not dedined!")


# print(rates_calculation)


# model_fun = output_parser['model']
# eff_species_list = output_parser['species_model']


# state_list = [0.5, 0.1]

# result = model_fun(state_list, rates_calculation)

# print(eff_species_list)
# print(result)



